<a href="https://colab.research.google.com/github/chavgova/My-AI/blob/master/emotion_recognition_15_male_8features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SPEECH RECOGNITION**

In [22]:
#this is the copy of another projecct and ill make changes to see how i can make it better 
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
from matplotlib.axis import Axis
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
import pandas as pd


In [19]:
from google.colab import drive
import os

path = '/content/drive/My Drive/My_AI/RawData'

mylist = []
mylist = os.listdir(path)

#print(mylist)	



In [20]:
print(mylist[4000:])	
print(len(mylist))

['YAF_door_neutral.wav', 'YAF_door_ps.wav', 'YAF_dog_disgust.wav', 'YAF_dodge_happy.wav', 'YAF_door_disgust.wav', 'YAF_dog_sad.wav', 'YAF_dog_fear.wav', 'YAF_dog_ps.wav', 'YAF_fit_happy.wav', 'YAF_fat_fear.wav', 'YAF_fit_angry.wav', 'YAF_five_disgust.wav', 'YAF_fit_neutral.wav', 'YAF_fall_happy.wav', 'YAF_five_angry.wav', 'YAF_fit_ps.wav', 'YAF_far_happy.wav', 'YAF_fit_fear.wav', 'YAF_fail_ps.wav', 'YAF_far_fear.wav', 'YAF_fat_disgust.wav', 'YAF_five_fear.wav', 'YAF_fat_ps.wav', 'YAF_far_sad.wav', 'YAF_far_ps.wav', 'YAF_far_disgust.wav', 'YAF_fat_sad.wav', 'YAF_fall_ps.wav', 'YAF_far_angry.wav', 'YAF_fat_angry.wav', 'YAF_fall_sad.wav', 'YAF_far_neutral.wav', 'YAF_five_happy.wav', 'YAF_fat_happy.wav', 'YAF_fall_neutral.wav', 'YAF_fall_angry.wav', 'YAF_fail_sad.wav', 'YAF_fit_sad.wav', 'YAF_fall_fear.wav', 'YAF_fall_disgust.wav', 'YAF_fat_neutral.wav', 'YAF_fit_disgust.wav', 'YAF_good_neutral.wav', 'YAF_goal_angry.wav', 'YAF_five_neutral.wav', 'YAF_goose_sad.wav', 'YAF_goal_fear.wav', 'Y

# LABLES & FEATURES

In [21]:
import re

feeling_list=[]
dataset = ''
count = 0
for item in mylist:
    file_label = item[6:-16]
    try:
      file_label = int(file_label)
      dataset = 'RAVDESS'
    except:
      if (item[:1] == 'Y') or (item[:1] == 'O'):
        file_label = re.split('_|\.', item)[2]
        dataset = 'TESS'
      else: 
        try:
          item = item[:-4]
          int(item[-3:])  
          dataset = 'SER_v4'
        except:
          dataset = 'SAVEE'

    if dataset == 'RAVDESS':
      if int(item[18:-4])%2==0:  #female
          if file_label == 1:
            feeling_list.append('female_neutral')
          elif file_label == 2:
            feeling_list.append('female_calm')
          elif file_label == 3:
            feeling_list.append('female_joy')
          elif file_label == 4:
            feeling_list.append('female_sadness')
          elif file_label == 5:
            feeling_list.append('female_anger')
          elif file_label == 6:
            feeling_list.append('female_fear')
          elif file_label == 7:
            feeling_list.append('female_disgust')
          elif file_label == 8:
            feeling_list.append('female_surprise')
      else:
          if file_label== 1:
            feeling_list.append('male_neutral')
          elif file_label == 2:
            feeling_list.append('male_calm')
          elif file_label == 3:
            feeling_list.append('male_joy')
          elif file_label == 4:
            feeling_list.append('male_sadness')
          elif file_label == 5:
            feeling_list.append('male_anger')
          elif file_label == 6:
            feeling_list.append('male_fear')
          elif file_label == 7:
            feeling_list.append('male_disgust')
          elif file_label == 8:
            feeling_list.append('male_surprise')

    elif dataset == 'TESS':
      if file_label == 'neutral': feeling_list.append('female_neutral')
      elif file_label == 'angry': feeling_list.append('female_anger')
      elif file_label == 'disgust': feeling_list.append('female_disgust')
      elif file_label == 'ps': feeling_list.append('female_surprise')
      elif file_label == 'happy': feeling_list.append('female_joy')
      elif file_label == 'sad': feeling_list.append('female_sadness')
      elif file_label == 'fear': feeling_list.append('female_fear')

    elif dataset == 'SER_v4':
      if int(item[-3:])%2 == 1: # male
        file_label = item[:-3]
        if file_label == 'neutral': feeling_list.append('male_neutral')
        elif file_label == 'anger': feeling_list.append('male_anger')
        elif file_label == 'disgust': feeling_list.append('male_disgust')
        elif file_label == 'surprise': feeling_list.append('male_surprise')
        elif file_label == 'happy': feeling_list.append('male_joy')
        elif file_label == 'sad': feeling_list.append('male_sadness')
        elif file_label == 'fear': feeling_list.append('male_fear')
      else:
        file_label = item[:-3]
        if file_label == 'neutral': feeling_list.append('female_neutral')
        elif file_label == 'anger': feeling_list.append('female_anger')
        elif file_label == 'disgust': feeling_list.append('female_disgust')
        elif file_label == 'surprise': feeling_list.append('female_surprise')
        elif file_label == 'happy': feeling_list.append('female_joy')
        elif file_label == 'sad': feeling_list.append('female_sadness')
        elif file_label == 'fear': feeling_list.append('female_fear')
      
    elif dataset == 'SAVEE':
      if item[:1]=='a':
        feeling_list.append('male_anger')
      elif item[:1]=='f':
        feeling_list.append('male_fear')
      elif item[:1]=='h':
        feeling_list.append('male_joy')
      elif item[:1]=='n':
        feeling_list.append('male_neutral')
      elif item[:2]=='sa':
        feeling_list.append('male_sadness')
      elif item[:2]=='su':
        feeling_list.append('male_surprise')
      elif item[:1]=='d':
        feeling_list.append('male_disgust')


In [22]:
import pandas as pd

labels = pd.DataFrame(feeling_list)
labels #[2600:2700]    

,0
0,male_sadness
1,male_sadness
2,male_sadness
3,female_sadness
4,male_sadness
...,...
7193,male_fear
7194,male_anger
7195,female_calm
7196,female_sadness


Getting the features of audio files using librosa

In [23]:
import librosa
import numpy as np

def reshape_feature(arr):   # reshapes to 10 values per feature
  shape_arr = arr.shape[0]
  r = shape_arr%10
  arr = arr[:(len(arr)-r)]
  d = int(shape_arr/10)
  arr = np.mean(arr.reshape(-1, d), axis=1) 
  return arr


def extract_feature(my_file, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    zcr = kwargs.get("zero_crossings")
    rolloff = kwargs.get("rolloff")
    centroids = kwargs.get("centroids")
    rms = kwargs.get("rms")

    X, sample_rate = librosa.core.load(my_file)

    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))  # 40 values 
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))  # 12 values 
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))  # 128 values 
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate),axis=0)
        contrast = reshape_feature(contrast)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate),axis=0)
        tonnetz = reshape_feature(tonnetz) 
        result = np.hstack((result, tonnetz)) 
    if rolloff:    
        rolloff = np.mean(librosa.feature.spectral_rolloff(X+0.01, sr=sample_rate),axis=0)
        rolloff = reshape_feature(rolloff)
        result = np.hstack((result, rolloff)) 
    if centroids:    
        centroids = np.mean(librosa.feature.spectral_centroid(X, sr=sample_rate),axis=0)
        centroids = reshape_feature(centroids)
        result = np.hstack((result, centroids)) 
    if rms:    
        rms = np.mean(librosa.feature.rms(X),axis=0)
        rms = reshape_feature(rms)
        result = np.hstack((result, rms)) 

    return result

f = os.fspath('/content/drive/MyDrive/My_AI/RawData/03-01-04-01-02-02-02.wav')
a = extract_feature(f, mel=True, mfcc=True, contrast=True, chroma=True, tonnetz=True, rolloff=True, centroids=True, rms=True)
print(a, a.shape)

[-6.77465229e+02  4.48900854e+01 -6.03083280e+00  7.03832029e+00
 -3.46486574e+00 -3.58694775e+00 -7.29098775e+00 -9.73034891e+00
 -5.93385890e+00 -3.85816179e-01 -6.36726774e+00 -3.24170919e+00
 -6.40472281e+00 -5.47514113e-01 -6.52374683e+00 -2.66422793e+00
 -3.61283989e+00 -2.82269210e+00 -3.81976559e+00 -3.16984448e+00
 -2.29157928e+00 -5.05138406e+00 -4.90422179e+00 -2.50711803e+00
 -2.35515434e+00 -2.73220500e+00  3.51941874e-01  2.13792434e+00
  3.80361651e+00  6.28113467e+00  5.77018297e+00  1.09428833e+01
  1.11916238e+01  1.07371013e+01  7.67549372e+00  5.60871802e+00
 -1.72933670e+00  3.23576154e-01  2.14877003e-01  3.10003708e+00
  4.97549440e-01  4.51846514e-01  5.18454691e-01  5.36993544e-01
  4.62111868e-01  4.84030333e-01  5.29520098e-01  6.54284557e-01
  7.16984774e-01  6.06158096e-01  5.62067607e-01  5.50278816e-01
  1.33784213e-05  1.03905941e-05  8.90924740e-06  5.52197920e-06
  3.36749410e-06  6.90607039e-05  5.00928071e-03  7.79857156e-02
  8.00177249e-02  2.81499

In [29]:
data_frame = pd.DataFrame(columns=['all_features'])
bookmark=0

for index,y in enumerate(mylist):
      all_features_ndarray = extract_feature('/content/drive/My Drive/My_AI/RawData/'+ y, mel=True, mfcc=True, contrast=True, chroma=True, tonnetz=True, rolloff=True, centroids=True, rms=True)
      data_frame.loc[bookmark] = [all_features_ndarray]
      bookmark=bookmark+1   



In [30]:
data_frame

,all_features
0,"[-677.0126067213283, 68.12888721970869, 1.2261..."
1,"[-557.0776311573632, 37.73133971497115, -25.69..."
2,"[-651.3068149027393, 28.174296589333228, -22.3..."
3,"[-598.479799297571, 83.87693278618099, -4.3676..."
4,"[-620.5454399369338, 63.076147215657386, -4.55..."
...,...
7193,"[-714.1079051201358, 61.88775583094703, 7.4510..."
7194,"[-581.1174602895977, 68.77734003400334, -0.720..."
7195,"[-727.8029769587318, 47.48866941475508, -3.987..."
7196,"[-714.3190524203625, 49.878237640280844, -4.65..."


In [31]:
data_frame = pd.DataFrame(data_frame['all_features'].values.tolist())
data_frame[:10]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229
0,-677.012607,68.128887,1.226144,13.246095,1.681556,0.729557,-13.641576,-3.576321,-8.595147,-4.046197,4.691763,-6.023820,1.683967,-0.069172,-5.879847,0.592705,-6.601959,-6.193829,0.361195,-2.431681,-6.519171,-0.058899,-5.800941,-3.980719,-2.219832,-0.920634,-2.284002,-1.727077,-1.160240,-1.174291,-2.115416,0.928084,1.233274,5.737881,-0.308995,2.263868,0.248554,6.760577,9.820166,17.682036,...,-0.002758,0.024366,-0.014609,-0.017491,0.038692,-0.014774,0.009052,-0.014159,-0.007447,0.042028,10.766602,10.766602,2483.118318,3031.648335,2326.719264,1352.058491,1034.160413,592.163086,10.766602,10.766602,4141.946889,3960.300525,3169.100261,2211.468405,1750.150643,1249.341416,1107.307061,874.524551,3054.225012,3151.287050,1.275939e-05,1.837397e-05,0.001840,0.006945,0.006974,0.007772,0.005745,0.004917,0.000101,0.000083
1,-557.077631,37.731340,-25.690923,2.721265,-8.469222,-16.783412,-11.141711,-16.858932,-20.300446,1.526086,-9.046842,-4.551558,-5.803873,-6.084743,0.324458,-1.555424,-1.621261,13.536664,13.608310,16.614511,16.819150,12.947100,1.430434,4.995464,-5.904416,3.786624,2.862549,2.213109,-3.014830,-2.941496,0.153834,5.607506,1.279606,-2.695713,-3.278017,1.666410,3.034963,-0.849230,3.336941,10.858757,...,0.005783,0.007931,-0.047082,-0.038384,0.032807,-0.049675,-0.022629,-0.083113,-0.007013,0.009937,10.766602,99.462891,4120.532227,4847.534180,3142.309570,2521.435547,3741.650391,1993.359375,87.158203,10.766602,4724.091019,3319.011026,2992.454686,3003.583954,1871.331079,1582.119839,2224.851651,1357.369560,3383.279798,4125.006267,3.837804e-06,9.196399e-05,0.009108,0.012666,0.030902,0.023994,0.014636,0.020256,0.000184,0.000019
2,-651.306815,28.174297,-22.387151,-3.099898,-21.895704,-20.684444,-6.937055,-22.691892,-16.077010,-2.648343,-13.845468,-4.334402,-9.532113,-4.433517,-8.319336,-6.686531,5.342289,12.560371,14.379339,23.704241,15.249805,10.304655,-0.133664,1.355806,-6.263078,1.910011,-2.980607,-1.288093,-8.861574,-4.063428,-4.258260,2.463083,-3.307999,-5.789722,-2.006577,1.961835,-0.318634,0.151110,3.371160,9.975064,...,-0.000333,0.001658,-0.053201,0.002287,-0.001398,-0.059190,-0.053723,-0.039287,-0.031767,-0.000971,10.766602,746.810636,4213.656339,3384.628018,2375.014426,4060.476962,1512.218129,1479.428933,108.644798,10.766602,1005.430834,1924.243185,2937.317570,2505.250228,1780.086072,2723.276573,1538.029538,1392.617717,3731.779374,4747.144549,6.222113e-07,9.804880e-05,0.006586,0.007667,0.008629,0.006623,0.006743,0.007385,0.000361,0.000003
3,-598.479799,83.876933,-4.367654,17.879762,1.207946,-22.144229,-9.396517,-0.343920,-17.230260,2.128921,-0.053539,-7.968760,-5.630409,0.340969,-4.279300,-0.842008,-4.132873,-3.515869,-1.105263,-3.192063,-7.741529,-2.782670,-8.780764,-1.375007,-4.971819,1.227432,-5.522446,-0.596580,-2.901904,-2.884426,-0.602703,-0.587759,0.122286,2.224288,3.667640,1.273863,-0.666005,2.981427,6.911973,14.680708,...,0.037110,0.025484,-0.033734,0.003408,0.028944,-0.016853,0.020251,-0.003134,-0.024512,-0.012597,10.766602,657.698922,3549.233611,2963.155995,2875.150730,1704.867952,2240.389351,805.622665,501.817255,10.766602,3090.152980,2984.077314,2357.822539,1973.434529,1943.695531,1239.052240,1601.832606,852.554178,1158.985943,2473.661473,6.118279e-05,2.461580e-04,0.006546,0.011865,0.007853,0.008946,0.006515,0.010970,0.004595,0.000020
4,-620.545440,63.076147,-4.554053,12.682791,0.132151,1.816087,-10.988464,-12.289517,-18.790218,-3.880212,-2.682088,-8.190950,-5.754832,-9.954274,-0.609016,2.046038,-12.034189,-4.354440,-3.916669,-7.596240,-10.865715,-2.858283,-8.838741,-8.424173,-3.157844,-6.277223,-5.993330,-3.999957,-6.670746,-3.156587,-4.428938,-0.825791,0.930508,-1.368441,-1.244825,0.040529,0.498666,3.362422,8.152221,14.178395,...,-0.004728,0.003

In [32]:
data_frame_labels = pd.concat([data_frame,labels], axis=1)
data_frame_labels = data_frame_labels.rename(index=str, columns={"0": "label"})
data_frame_labels    #print

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,0
0,-677.012607,68.128887,1.226144,13.246095,1.681556,0.729557,-13.641576,-3.576321,-8.595147,-4.046197,4.691763,-6.023820,1.683967,-0.069172,-5.879847,0.592705,-6.601959,-6.193829,0.361195,-2.431681,-6.519171,-0.058899,-5.800941,-3.980719,-2.219832,-0.920634,-2.284002,-1.727077,-1.160240,-1.174291,-2.115416,0.928084,1.233274,5.737881,-0.308995,2.263868,0.248554,6.760577,9.820166,17.682036,...,0.024366,-0.014609,-0.017491,0.038692,-0.014774,0.009052,-0.014159,-0.007447,0.042028,10.766602,10.766602,2483.118318,3031.648335,2326.719264,1352.058491,1034.160413,592.163086,10.766602,10.766602,4141.946889,3960.300525,3169.100261,2211.468405,1750.150643,1249.341416,1107.307061,874.524551,3054.225012,3151.287050,1.275939e-05,1.837397e-05,0.001840,0.006945,0.006974,0.007772,0.005745,0.004917,0.000101,0.000083,male_sadness
1,-557.077631,37.731340,-25.690923,2.721265,-8.469222,-16.783412,-11.141711,-16.858932,-20.300446,1.526086,-9.046842,-4.551558,-5.803873,-6.084743,0.324458,-1.555424,-1.621261,13.536664,13.608310,16.614511,16.819150,12.947100,1.430434,4.995464,-5.904416,3.786624,2.862549,2.213109,-3.014830,-2.941496,0.153834,5.607506,1.279606,-2.695713,-3.278017,1.666410,3.034963,-0.849230,3.336941,10.858757,...,0.007931,-0.047082,-0.038384,0.032807,-0.049675,-0.022629,-0.083113,-0.007013,0.009937,10.766602,99.462891,4120.532227,4847.534180,3142.309570,2521.435547,3741.650391,1993.359375,87.158203,10.766602,4724.091019,3319.011026,2992.454686,3003.583954,1871.331079,1582.119839,2224.851651,1357.369560,3383.279798,4125.006267,3.837804e-06,9.196399e-05,0.009108,0.012666,0.030902,0.023994,0.014636,0.020256,0.000184,0.000019,male_sadness
2,-651.306815,28.174297,-22.387151,-3.099898,-21.895704,-20.684444,-6.937055,-22.691892,-16.077010,-2.648343,-13.845468,-4.334402,-9.532113,-4.433517,-8.319336,-6.686531,5.342289,12.560371,14.379339,23.704241,15.249805,10.304655,-0.133664,1.355806,-6.263078,1.910011,-2.980607,-1.288093,-8.861574,-4.063428,-4.258260,2.463083,-3.307999,-5.789722,-2.006577,1.961835,-0.318634,0.151110,3.371160,9.975064,...,0.001658,-0.053201,0.002287,-0.001398,-0.059190,-0.053723,-0.039287,-0.031767,-0.000971,10.766602,746.810636,4213.656339,3384.628018,2375.014426,4060.476962,1512.218129,1479.428933,108.644798,10.766602,1005.430834,1924.243185,2937.317570,2505.250228,1780.086072,2723.276573,1538.029538,1392.617717,3731.779374,4747.144549,6.222113e-07,9.804880e-05,0.006586,0.007667,0.008629,0.006623,0.006743,0.007385,0.000361,0.000003,male_sadness
3,-598.479799,83.876933,-4.367654,17.879762,1.207946,-22.144229,-9.396517,-0.343920,-17.230260,2.128921,-0.053539,-7.968760,-5.630409,0.340969,-4.279300,-0.842008,-4.132873,-3.515869,-1.105263,-3.192063,-7.741529,-2.782670,-8.780764,-1.375007,-4.971819,1.227432,-5.522446,-0.596580,-2.901904,-2.884426,-0.602703,-0.587759,0.122286,2.224288,3.667640,1.273863,-0.666005,2.981427,6.911973,14.680708,...,0.025484,-0.033734,0.003408,0.028944,-0.016853,0.020251,-0.003134,-0.024512,-0.012597,10.766602,657.698922,3549.233611,2963.155995,2875.150730,1704.867952,2240.389351,805.622665,501.817255,10.766602,3090.152980,2984.077314,2357.822539,1973.434529,1943.695531,1239.052240,1601.832606,852.554178,1158.985943,2473.661473,6.118279e-05,2.461580e-04,0.006546,0.011865,0.007853,0.008946,0.006515,0.010970,0.004595,0.000020,female_sadness
4,-620.545440,63.076147,-4.554053,12.682791,0.132151,1.816087,-10.988464,-12.289517,-18.790218,-3.880212,-2.682088,-8.190950,-5.754832,-9.954274,-0.609016,2.046038,-12.034189,-4.354440,-3.916669,-7.596240,-10.865715,-2.858283,-8.838741,-8.424173,-3.157844,-6.277223,-5.993330,-3.999957,-6.670746,-3.156587,-4.428938,-0.825791,0.930508,-1.368441,-1.244825,0.040529,0.498666,3.362422,8.152221,14.178395,...,0

In [33]:
from sklearn.utils import shuffle
data_frame_labels = shuffle(data_frame_labels)
data_frame_labels

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,0
6911,-560.092745,55.183104,-11.826187,5.802595,2.109797,-1.559523,-11.414464,-7.296623,-14.805032,-7.140440,2.974121,-7.601741,-1.929785,-1.682946,-6.232723,1.044203,-7.599880,-5.628976,-2.752589,-4.389025,-6.402810,-3.898047,-3.678719,-4.301682,-2.361739,1.222361,-4.370849,1.017799,-1.008354,-2.208833,-1.023214,1.997225,0.728315,1.397582,-0.546476,4.313166,4.153224,5.380476,1.551094,0.088582,...,0.005524,0.000834,-0.009672,-0.001706,-0.022646,-0.011870,-0.011310,0.031405,0.047394,10.766602,10.766602,665.375977,4911.005859,3262.280273,2612.695312,2647.148438,1346.542969,19.379883,10.766602,3392.619096,2900.006559,3500.690785,2921.404940,1960.438437,1513.526379,1530.314947,1372.805050,1689.798113,4094.031697,3.524949e-05,4.348073e-05,0.000381,0.017277,0.023828,0.018989,0.017803,0.006876,0.000665,0.000019,male_joy
1529,-550.861058,32.147106,-23.562386,4.293873,-13.721061,-12.340165,-7.485299,-20.391139,-14.558584,-7.212806,-11.859553,-5.956253,-10.710912,0.832868,-9.358627,1.834055,-0.829754,7.276259,7.808383,12.318950,11.584263,11.121283,1.634918,-4.339033,-7.409732,-2.067579,-1.528417,-0.166985,-7.229870,-7.815880,-1.587016,5.476586,0.647048,-3.974973,-2.709758,1.394385,-1.302132,-1.071307,3.079398,9.096373,...,0.025525,-0.006145,-0.051515,-0.020213,0.005809,-0.053390,-0.071485,-0.017913,-0.000703,69.384766,1199.877930,4821.044922,4581.787109,5543.603516,2515.795898,4526.757812,1416.406250,263.781738,10.766602,3528.415514,3269.306829,4157.657578,2714.577733,3392.142985,1627.267904,2549.713978,1307.280191,3415.418777,4042.665051,3.744098e-05,2.150233e-04,0.007493,0.021030,0.015569,0.020805,0.022013,0.021830,0.000206,0.000023,female_fear
2709,-437.588277,53.607519,-26.603353,16.846902,3.846265,-5.152013,1.321800,-7.672410,-14.682791,7.359875,-8.463136,-1.345556,6.410548,-6.132278,-4.828611,1.782513,-4.703288,-1.514069,-5.150079,0.215666,-5.041509,0.845736,-4.444120,-0.718559,-2.325182,1.721723,-3.688140,1.123921,-3.847166,-1.937895,-3.254891,0.471214,1.787281,3.213657,1.045712,1.827670,3.775180,7.693228,10.895229,12.648948,...,0.005101,-0.005439,-0.003775,0.005856,0.014475,-0.027634,-0.005301,0.115614,0.044683,2847.276722,2899.152166,5084.282892,3926.383833,3248.087935,3537.318004,3236.342551,1690.356445,14.192338,10.766602,3804.647552,3762.125201,2983.711896,2236.003527,1902.274788,1887.353160,1661.717557,1575.510312,310.186246,1330.727003,2.785818e-04,4.605422e-04,0.025523,0.068294,0.048495,0.057808,0.075951,0.007180,0.006055,0.000546,male_anger
638,-436.373311,126.102626,28.414725,42.019648,11.559585,-0.970433,-14.796186,-1.437694,4.202753,0.227801,1.227511,2.112023,1.692150,0.244891,8.593436,2.313012,-0.228346,4.077587,-2.179745,-1.586641,2.601010,1.746958,0.260893,-2.840402,-0.518300,-0.123019,-0.515713,-0.386747,0.139475,0.747742,-0.505346,1.382192,-0.938106,0.419076,1.503124,-1.317595,1.169955,2.248309,2.471575,2.557306,...,0.009294,-0.035536,0.039106,0.011853,0.015370,0.018715,0.001945,-0.000076,0.012555,393.683127,1366.422172,1931.434698,1241.435971,550.032906,257.930324,2616.284180,1798.490574,1056.999406,117.496391,345.820286,841.185960,1026.980956,608.553182,392.596226,304.215590,1060.925089,927.900545,536.967752,179.326315,4.036814e-02,7.700963e-02,0.082951,0.064330,0.053711,0.053106,0.077386,0.060571,0.057789,0.043479,male_neutral
3813,-393.394831,74.895783,23.355105,38.045369,2.128537,-4.666225,-5.849078,-11.315678,0.749105,-11.902566,-6.421195,5.046442,-9.504804,10.281113,-7.351656,-2.371447,1.023657,-2.079523,-10.062383,-2.055895,-7.781969,-7.632018,-4.145398,2.181581,-1.240011,8.556646,3.038294,1.372998,-1.207662,-0.614657,-0.839577,11.978713,7.493061,7.856309,-0.201933,1.696574,0.722453,5.056321,7.454724,8.254524,...,0.015509,0.057290,0.01526

# SAVE DATASET FEATURES AND LABELS 

In [34]:
import pickle

with open('/content/drive/My Drive/My_AI/datasets_RAVDESS-TESS-SAVEE-SER_v4_features&labels_8features.pkl', 'wb') as f:
  pickle.dump(data_frame_labels, f)


# LOAD DATASET FEATURES AND LABELS 

In [23]:
import pickle

with open('/content/drive/My Drive/My_AI/datasets_RAVDESS-TESS-SAVEE-SER_v4_features&labels_8features.pkl', 'rb') as f:
  data_frame_labels = pickle.load(f)


# Dividing the data into test and train

In [24]:
import numpy as np

data_frame_labels.rename(columns={'0': 'lables'}, inplace=True)
data_frame_labels = data_frame_labels.dropna(axis=1) 
columns_arr = [i for i in range(230)]
columns_arr.append('lables')
data_frame_labels.columns = columns_arr

print(data_frame_labels)

data_frame_labels = data_frame_labels[data_frame_labels.lables != 'female_neutral']
data_frame_labels = data_frame_labels[data_frame_labels.lables != 'female_calm']
data_frame_labels = data_frame_labels[data_frame_labels.lables != 'female_fear']
data_frame_labels = data_frame_labels[data_frame_labels.lables != 'female_surprise']
data_frame_labels = data_frame_labels[data_frame_labels.lables != 'female_joy']
data_frame_labels = data_frame_labels[data_frame_labels.lables != 'female_sadness']
data_frame_labels = data_frame_labels[data_frame_labels.lables != 'female_anger']
data_frame_labels = data_frame_labels[data_frame_labels.lables != 'female_disgust']



print(data_frame_labels)

data_frame_labels_set = np.random.rand(len(data_frame_labels)) < 0.8

train = data_frame_labels[data_frame_labels_set]
test = data_frame_labels[~data_frame_labels_set]





               0           1          2  ...       228       229          lables
6911 -560.092745   55.183104 -11.826187  ...  0.000665  0.000019        male_joy
1529 -550.861058   32.147106 -23.562386  ...  0.000206  0.000023     female_fear
2709 -437.588277   53.607519 -26.603353  ...  0.006055  0.000546      male_anger
638  -436.373311  126.102626  28.414725  ...  0.057789  0.043479    male_neutral
3813 -393.394831   74.895783  23.355105  ...  0.040820  0.009691  female_sadness
...          ...         ...        ...  ...       ...       ...             ...
349  -533.044708   47.122457  -6.093841  ...  0.000055  0.000033   male_surprise
2108 -670.996166   52.998476  10.554924  ...  0.000023  0.000023    male_disgust
6593 -529.560500   54.540417 -17.776009  ...  0.000549  0.000325     female_fear
344  -525.627557   44.731324   0.738930  ...  0.000175  0.000071   male_surprise
7176 -450.385487   45.263907 -20.873818  ...  0.000259  0.000092       male_fear

[7198 rows x 231 columns]
 

In [7]:
test[0:20]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,lables
6911,-560.092745,55.183104,-11.826187,5.802595,2.109797,-1.559523,-11.414464,-7.296623,-14.805032,-7.140440,2.974121,-7.601741,-1.929785,-1.682946,-6.232723,1.044203,-7.599880,-5.628976,-2.752589,-4.389025,-6.402810,-3.898047,-3.678719,-4.301682,-2.361739,1.222361,-4.370849,1.017799,-1.008354,-2.208833,-1.023214,1.997225,0.728315,1.397582,-0.546476,4.313166,4.153224,5.380476,1.551094,0.088582,...,0.005524,0.000834,-0.009672,-0.001706,-0.022646,-0.011870,-0.011310,0.031405,0.047394,10.766602,10.766602,665.375977,4911.005859,3262.280273,2612.695312,2647.148438,1346.542969,19.379883,10.766602,3392.619096,2900.006559,3500.690785,2921.404940,1960.438437,1513.526379,1530.314947,1372.805050,1689.798113,4094.031697,0.000035,0.000043,0.000381,0.017277,0.023828,0.018989,0.017803,0.006876,0.000665,0.000019,male_joy
62,-634.694864,70.133681,-9.407845,12.772225,2.482182,-3.252000,-14.044456,-5.083026,-9.419722,-6.474363,5.095733,-6.692916,-0.201481,2.544239,-7.139405,0.939528,-2.526470,-7.368815,0.062660,-0.536732,-6.410064,-0.381074,-4.843600,-4.452889,-2.074956,1.680046,-3.014430,-1.752561,-1.203835,0.600734,-1.273337,3.167817,0.614886,3.607591,-1.107325,2.331228,1.887489,6.942906,9.828703,14.272437,...,0.023735,-0.021908,-0.014016,0.008761,-0.022940,0.023859,-0.006902,-0.024620,0.004903,10.766602,10.766602,2022.924805,3146.240234,2541.516113,1604.821777,1499.548340,1424.780273,135.180664,10.766602,3932.851900,4449.789333,3261.312628,2272.230317,1686.920020,1200.249049,1270.620514,1059.779416,2228.048313,3401.588213,0.000028,0.000011,0.001803,0.006563,0.010580,0.012073,0.008265,0.012238,0.000514,0.000036,male_sadness
2098,-550.856433,51.140533,-7.205667,14.601712,9.071430,-10.928430,-1.568134,-4.587918,-12.000161,1.186417,-2.219359,-5.469356,-3.974080,-2.361698,-4.345183,-1.294386,-4.418996,0.525529,-4.343213,-1.544888,-3.617492,1.347607,-4.634121,-1.695529,-3.196757,-1.384517,-2.837618,-0.937316,-0.820356,-0.481664,-2.740986,-1.612465,-3.613873,-1.383992,-1.827862,-0.832924,-2.943780,-0.990474,-3.205579,-1.610765,...,0.006988,0.010626,-0.022263,-0.013112,-0.031726,-0.029789,-0.026435,-0.005862,0.005252,10.766602,10.766602,1022.827148,4450.195312,4909.570312,1836.064453,3534.316406,1652.314453,10.766602,10.766602,4125.470214,3857.201967,3430.167697,2720.232405,3237.802359,1368.598948,1947.191718,1820.797801,3157.130353,3598.945024,0.000010,0.000015,0.003516,0.021566,0.025538,0.015087,0.015508,0.007481,0.000032,0.000016,male_disgust
6546,-535.864691,48.126324,3.847470,17.641615,-0.378202,-14.825413,-1.480320,0.196073,-9.394686,-3.859728,-1.258181,-3.503633,-2.204695,-0.357789,-5.204171,2.682086,-6.988669,-0.112605,-2.146501,-2.981284,-6.446181,-0.610841,-5.770918,-2.491036,-1.625970,0.172777,-1.412235,3.269903,-1.501642,0.278286,-2.539713,2.101721,1.975832,3.787471,3.262097,2.767771,0.187457,3.823706,2.441751,3.813948,...,0.022753,0.023606,0.013656,0.015624,-0.013078,-0.000564,0.000786,0.001341,-0.011063,10.766602,10.766602,3722.710823,5060.302734,5093.235869,3793.643727,3378.179573,2716.983571,253.965131,10.766602,3678.337800,4770.085383,3930.569305,3064.042879,2787.359172,2152.394731,2136.643677,1623.465450,2649.465832,2927.639328,0.000006,0.000011,0.010379,0.017896,0.019091,0.009384,0.013200,0.021073,0.000421,0.000120,male_disgust
5915,-270.733725,83.003577,6.052960,33.817712,-20.038983,-4.812151,-12.862700,-2.405926,-14.516592,-4.941371,-6.710842,-5.734905,-5.430515,-6.956309,2.468767,-3.239425,-3.058268,-1.172504,-2.365001,-2.038726,5.545191,3.986789,4.696110,5.878036,4.350276,2.854161,2.965956,2.896474,0.947407,2.905728,2.815319,3.088644,1.637376,-0.880481,2.240374,1.715082,1.630105,4.639538,2.217030,2.520463,...,-0.011758,-0.001779,-0.000401,-0.004717,-0.0095

In [25]:
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]

In [26]:
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]
testlabel

,lables
2709,male_anger
5915,male_surprise
740,male_sadness
5952,male_disgust
5834,male_joy
...,...
2105,male_disgust
5752,male_neutral
293,male_surprise
6510,male_joy


In [27]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder


X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

#print(y_test)

lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_test

In [ ]:
X_test

Changing dimension for CNN model

In [28]:
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)
print(x_testcnn)

[[[-4.37588277e+02]
  [ 5.36075189e+01]
  [-2.66033533e+01]
  ...
  [ 7.18023907e-03]
  [ 6.05468312e-03]
  [ 5.46439434e-04]]

 [[-2.70733725e+02]
  [ 8.30035766e+01]
  [ 6.05295981e+00]
  ...
  [ 2.62286782e-01]
  [ 1.12696126e-01]
  [ 1.40194669e-02]]

 [[-5.95133997e+02]
  [ 6.34848667e+01]
  [-5.03812539e+00]
  ...
  [ 3.90348863e-03]
  [ 3.51834181e-03]
  [ 8.14123196e-04]]

 ...

 [[-7.45083821e+02]
  [ 6.70981154e+01]
  [ 1.78463003e+01]
  ...
  [ 5.30276229e-05]
  [ 3.28873575e-05]
  [ 6.46265180e-05]]

 [[-4.51976134e+02]
  [ 4.91341557e+01]
  [-1.20061298e+01]
  ...
  [ 5.03075011e-02]
  [ 4.62312717e-03]
  [ 5.30240592e-04]]

 [[-6.17143677e+02]
  [ 4.77144971e+01]
  [-1.11960267e+00]
  ...
  [ 2.10831291e-03]
  [ 1.80933112e-05]
  [ 2.12334235e-05]]]


# **MODEL**

In [29]:
model = Sequential()

model.add(Conv1D(256, 5,padding='same', input_shape=(230,1)))
model.add(Activation('relu'))

model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))

model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(8))

model.add(Activation('sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.0001) ###

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 230, 256)          1536      
_________________________________________________________________
activation_7 (Activation)    (None, 230, 256)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 230, 128)          163968    
_________________________________________________________________
activation_8 (Activation)    (None, 230, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 230, 128)          82048     
_________________________________________________________________
activation_9 (Activation)    (None, 230, 128)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 230, 128)         

In [30]:
model.compile(loss= 'categorical_crossentropy', optimizer = opt, metrics=['accuracy'])

In [31]:
cnnhistory = model.fit(x_traincnn, y_train, batch_size = 32, epochs = 50, validation_data = (x_testcnn, y_test))

Epoch 1/50
62/62 [==============================] - 33s 536ms/step - loss: 2.1876 - accuracy: 0.1475 - val_loss: 2.0445 - val_accuracy: 0.1377
Epoch 2/50
62/62 [==============================] - 33s 534ms/step - loss: 2.0384 - accuracy: 0.1506 - val_loss: 2.0330 - val_accuracy: 0.1377
Epoch 3/50
62/62 [==============================] - 33s 533ms/step - loss: 2.0369 - accuracy: 0.1501 - val_loss: 2.0429 - val_accuracy: 0.1377
Epoch 4/50
62/62 [==============================] - 33s 536ms/step - loss: 2.0429 - accuracy: 0.1501 - val_loss: 2.0408 - val_accuracy: 0.1377
Epoch 5/50
17/62 [=======>......................] - ETA: 21s - loss: 2.0191 - accuracy: 0.1213

KeyboardInterrupt: ignored

# **PLOTTING**

In [ ]:
plt.figure()
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['loss', 'val loss'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(9,5))

plt.plot(cnnhistory.history['loss'], 'm', linewidth=3)
plt.plot(cnnhistory.history['val_loss'], 'b', linewidth=3)
plt.legend(['Loss', 'Validation Loss'], fontsize=13)
plt.xlabel('epochs')
plt.ylabel('loss', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6), frameon=True)

plt.plot(cnnhistory.history['accuracy'], 'g', linewidth=3)
plt.plot(cnnhistory.history['val_accuracy'], 'r', linewidth=3)
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy', fontsize=12)

plt.legend(['Accuracy', 'Validation Accuracy'], loc = 'upper left', fontsize=13)
plt.grid(True)
plt.show()

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="img_model.png",
    show_shapes=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

dot_img_file = '/content/drive/My Drive/My_AI/img_model_MALE_8features_.png'
tf.keras.utils.plot_model(model, to_file = dot_img_file, show_shapes=True)


# **SAVING THE MODEL**

In [ ]:
model_name = 'Emotion_Voice_Detection_CNN_model_15_MALE_8features.h5'
path = '/content/drive/My Drive/My_AI/MY MODELS/'
model_path = os.path.join(path, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [ ]:
import json
model_json = model.to_json()
with open("/content/drive/My Drive/My_AI/MY MODELS/model_15_MALE_8features.json", "w") as json_file:
    json_file.write(model_json)

# **LOADING THE MODEL**

In [ ]:
import tensorflow as tf
from keras.models import model_from_json
json_file = open('/content/drive/My Drive/My_AI/MY MODELS/model_15_MALE_8features.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/My_AI/MY MODELS/Emotion_Voice_Detection_CNN_model_15_MALE_8features.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
opt = tf.keras.optimizers.Adam(learning_rate=0.0001) ###
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = loaded_model.evaluate(x_testcnn, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

# **Predicting emotions on the test data**


In [ ]:
import pandas as pd

preds = loaded_model.predict(x_testcnn, batch_size=32, verbose=1)
preds1=preds.argmax(axis=1)

abc = preds1.astype(int).flatten()
predictions = (lb.inverse_transform((abc)))
preddf = pd.DataFrame({'predictedvalues': predictions})

actual=y_test.argmax(axis=1)

abc123 = actual.astype(int).flatten()
actualvalues = (lb.inverse_transform((abc123)))

actualdf = pd.DataFrame({'actualvalues': actualvalues})
finaldf = actualdf.join(preddf)

finaldf[10:70]


In [ ]:
finaldf.groupby('actualvalues').count()

In [ ]:
finaldf.groupby('predictedvalues').count()

In [ ]:
finaldf.to_csv('Predictions_15_MALE_8features.csv', index=False)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

classes = finaldf.actualvalues.unique()
classes.sort()    
print(classification_report(finaldf.actualvalues, finaldf.predictedvalues, target_names=classes))


In [ ]:
import seaborn as sns

def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Gender recode function
def gender(row):
    if row == 'male_disgust' or 'male_fear' or 'male_joy' or 'male_sadness' or 'male_surprise' or 'male_neutral' or 'male_anger' or 'male_calm':
        return 'male'

finaldf = pd.read_csv('Predictions_15_MALE_8features.csv')
classes = finaldf.actualvalues.unique()
classes.sort()    

# Confusion matrix 
c = confusion_matrix(finaldf.actualvalues, finaldf.predictedvalues)
#print(accuracy_score(finaldf.actualvalues, finaldf.predictedvalues))
print_confusion_matrix(c, class_names = classes)

# RECORD AUDIO 

In [ ]:
!pip install SpeechRecognition
!pip install pyttsx3
!pip install ffmpeg-python
!sudo apt-get install portaudio19-dev python-pyaudio python3-pyaudio

In [ ]:
"""
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin = True, pipe_stdout = True, pipe_stderr = True, quiet = True, overwrite_output = True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio_index = 1

In [ ]:
#audio, sr = get_audio()

In [ ]:
type(audio)

In [ ]:
from scipy.io.wavfile import write

samplerate = 44100; fs = 100
#t = np.linspace(0., 1., samplerate)
#amplitude = np.iinfo(np.int16).max
#data = amplitude * np.sin(2. * np.pi * fs * t)

audio_path = '/content/record_' + str(audio_index)
write(audio_path, samplerate, audio)
audio_index += 1

# **VOICE TEST**

In [ ]:
data, sampling_rate = librosa.load('/content/drive/My Drive/My_AI/Real Voice samples/recording_demo_5')

% pylab inline
import os
import pandas as pd
import librosa
import glob 
from librosa import display

plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

In [ ]:
X, sample_rate = librosa.load('/content/drive/My Drive/My_AI/Real Voice samples/recording_demo_5', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
sample_rate = np.array(sample_rate)

demo_file = os.fspath('/content/drive/My Drive/My_AI/Real Voice samples/recording_demo_5')
features_live  = extract_feature(demo_file, mel=True, mfcc=True, contrast=True, chroma=True, tonnetz=True)

In [ ]:
features_live = pd.DataFrame(data = features_live)
features_live = features_live.stack().to_frame().T

# **EMOTIONS**

In [ ]:
import torch
features_live_2d = np.expand_dims(features_live, axis=2)
live_preds = loaded_model.predict(features_live_2d, batch_size=32, verbose = 1)
#print(live_preds)


all = np.argsort(-live_preds, axis=1)[:, :8]
for i in all:
  print((lb.inverse_transform((i))))
 
 

print()
print()
best_n = np.argsort(-live_preds)[:, :3] # best_n = [* * *]

first_second = 0
second_third = 0

for n in best_n:
  k = n
  num = 1
  for k in n:   
    #print(live_preds[0][n])
    #print(k)
    #print(n)
    if num == 1: first_second = live_preds[0][k] / live_preds[0][n][1]  
    elif num == 2: second_third = live_preds[0][k] / live_preds[0][n][2]
    num += 1


for i in best_n:
  print((lb.inverse_transform((i))))



In [ ]:
for i in best_n:
  first_emo = lb.inverse_transform((i))[0][5:]
  second_emo = lb.inverse_transform((i))[1][5:]
  third_emo = lb.inverse_transform((i))[2][5:]

emotion = ''
intensity_emo = ''
mixed_emo = ''

if first_second >= 10000:
  emotion = first_emo

elif first_second >= 200:
  #mix intensity
  if first_emo == 'disgust' and second_emo == 'anger': intensity_emo = 'distant' 
  elif first_emo == 'anger' and second_emo == 'disgust': intensity_emo = 'irritation'  
  elif first_emo == 'disgust' and second_emo == 'sadness': intensity_emo = 'guilt'
  elif first_emo == 'sadness' and second_emo == 'disgust': intensity_emo = 'miserable'
  elif first_emo == 'sadness' and second_emo == 'fear': intensity_emo = 'anxious'
  elif first_emo == 'fear' and second_emo == 'sadness': intensity_emo = 'desperate'
  elif first_emo == 'fear' and second_emo == 'joy': intensity_emo = 'astonished'
  elif first_emo == 'joy' and second_emo == 'fear': intensity_emo = 'determined'
  #elif first_emo == '' and second_emo == '': intensity_emo = ''
  #elif first_emo == '' and second_emo == '': intensity_emo = ''

#elif second_third >= 100:
  #mix intensity

if first_second < 200: # in this case the first and second emos are close to equal 
  # mix
  if first_emo == 'disgust' or second_emo == 'disgust':
    if first_emo == 'sadness' or second_emo == 'sadness': emotion = 'remorse'
    elif first_emo == 'neutral' or second_emo == 'neutral': emotion = 'neutral disgust'
    elif first_emo == 'anger' or second_emo == 'anger': emotion = 'contempt'
    elif first_emo == 'fear' or second_emo == 'fear': emotion = 'shame'
    elif first_emo == 'joy' or second_emo == 'joy': emotion = 'morbidness'
    elif first_emo == 'surprise' or second_emo == 'surprise': emotion = 'unbelief'

  elif first_emo == 'fear' or second_emo == 'fear':
    if first_emo == 'neutral' or second_emo == 'neutral': emotion = 'neutral fear'
    elif first_emo == 'joy' or second_emo == 'joy': emotion = 'guilt'         #????
    elif first_emo == 'sadness' or second_emo == 'sadness': emotion = 'despair'
    elif first_emo == 'surprise' or second_emo == 'surprise': emotion = 'awe'
  
  elif first_emo == 'anger' or second_emo == 'anger':
    if first_emo == 'neutral' or second_emo == 'neutral': emotion = 'neutral anger'
    elif first_emo == 'joy' or second_emo == 'joy': emotion = 'pride' 
    elif first_emo == 'surprise' or second_emo == 'surprise': emotion = 'outrage'   

  elif first_emo == 'joy' or second_emo == 'joy':
    if first_emo == 'neutral' or second_emo == 'neutral': emotion = 'neutral joy'  
    elif first_emo == 'surprise' or second_emo == 'surprise': emotion = 'delight' 

  elif first_emo == 'sadness' or second_emo == 'sadness':
    if first_emo == 'neutral' or second_emo == 'neutral': emotion = 'neutral sadness'  
    elif first_emo == 'surprise' or second_emo == 'surprise': emotion = 'disappointment' 
    elif first_emo == 'anger' or second_emo == 'anger': emotion = 'envy'

else: emotion = first_emo

if second_third < 200:    # in this case the second and third emos are close to equal 
 # mix
  if third_emo == 'disgust' or second_emo == 'disgust':
    if third_emo == 'sadness' or second_emo == 'sadness': mixed_emo = 'remorse'
    elif third_emo == 'neutral' or second_emo == 'neutral': mixed_emo = 'neutral disgust'
    elif third_emo == 'anger' or second_emo == 'anger': mixed_emo = 'contempt'
    elif third_emo == 'fear' or second_emo == 'fear': mixed_emo = 'shame'
    elif third_emo == 'joy' or second_emo == 'joy': mixed_emo = 'morbidness'
    elif third_emo == 'surprise' or second_emo == 'surprise': mixed_emo = 'unbelief'

  elif third_emo == 'fear' or second_emo == 'fear':
    if third_emo == 'neutral' or second_emo == 'neutral': mixed_emo = 'neutral fear'
    elif third_emo == 'joy' or second_emo == 'joy': mixed_emo = 'guilt'         #????
    elif third_emo == 'sadness' or second_emo == 'sadness': mixed_emo = 'despair'
    elif third_emo == 'surprise' or second_emo == 'surprise': mixed_emo = 'awe'
  
  elif third_emo == 'anger' or second_emo == 'anger':
    if third_emo == 'neutral' or second_emo == 'neutral': mixed_emo = 'neutral anger'
    elif third_emo == 'joy' or second_emo == 'joy': mixed_emo = 'pride' 
    elif third_emo == 'surprise' or second_emo == 'surprise': mixed_emo = 'outrage'   

  elif third_emo == 'joy' or second_emo == 'joy':
    if third_emo == 'neutral' or second_emo == 'neutral': mixed_emo = 'neutral joy'  
    elif third_emo == 'surprise' or second_emo == 'surprise': mixed_emo = 'delight' 

  elif third_emo == 'sadness' or second_emo == 'sadness':
    if third_emo == 'neutral' or second_emo == 'neutral': mixed_emo = 'neutral sadness'  
    elif third_emo == 'surprise' or second_emo == 'surprise': mixed_emo = 'disappointment' 
    elif third_emo == 'anger' or second_emo == 'anger': mixed_emo = 'envy'


print(first_emo)
print('Main emotion:')
print(emotion)
print('Mixed emotion:')
print(mixed_emo)
print('Emotion by intensity:')
print(intensity_emo)